# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
housing= pd.read_csv("cal_housing_clean.csv")

In [5]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [6]:
housing.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [10]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [11]:
y_val = housing['medianHouseValue']

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()
scaler

MinMaxScaler(copy=True, feature_range=(0, 1))

In [15]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [20]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [21]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [22]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [23]:
import tensorflow as tf

**Continuous variables

In [26]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [27]:
feat_cols = [age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [36]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [37]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[10,10,10],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/jr/44cwffjj7nj68zjg2ngbdmjw0000gn/T/tmp85a_z68z', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [38]:
dnn_model.train(input_fn=input_func,steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/jr/44cwffjj7nj68zjg2ngbdmjw0000gn/T/tmp85a_z68z/model.ckpt.
INFO:tensorflow:loss = 8.63322e+11, step = 1
INFO:tensorflow:global_step/sec: 364.667
INFO:tensorflow:loss = 4.56162e+11, step = 101 (0.275 sec)
INFO:tensorflow:global_step/sec: 379.147
INFO:tensorflow:loss = 2.77959e+11, step = 201 (0.264 sec)
INFO:tensorflow:global_step/sec: 363.891
INFO:tensorflow:loss = 3.43829e+11, step = 301 (0.275 sec)
INFO:tensorflow:global_step/sec: 344.643
INFO:tensorflow:loss = 1.42838e+11, step = 401 (0.292 sec)
INFO:tensorflow:global_step/sec: 413.546
INFO:tensorflow:loss = 2.54161e+11, step = 501 (0.243 sec)
INFO:tensorflow:global_step/sec: 387.147
INFO:tensorflow:loss = 5.00958e+10, step = 601 (0.260 sec)
INFO:tensorflow:global_step/sec: 342.862
INFO:tensorflow:loss = 9.41401e+10, step = 701 (0.289 sec)
INFO:tensorflow:global_step/sec: 331.284
INFO:tensorflow:loss = 1.61455e+11, step = 801 (0.3

INFO:tensorflow:loss = 7.60245e+10, step = 8001 (0.226 sec)
INFO:tensorflow:global_step/sec: 295.378
INFO:tensorflow:loss = 1.6151e+11, step = 8101 (0.335 sec)
INFO:tensorflow:global_step/sec: 450.294
INFO:tensorflow:loss = 1.2021e+11, step = 8201 (0.222 sec)
INFO:tensorflow:global_step/sec: 341.849
INFO:tensorflow:loss = 1.93055e+11, step = 8301 (0.296 sec)
INFO:tensorflow:global_step/sec: 343.14
INFO:tensorflow:loss = 3.69608e+10, step = 8401 (0.287 sec)
INFO:tensorflow:global_step/sec: 389.891
INFO:tensorflow:loss = 8.42481e+10, step = 8501 (0.261 sec)
INFO:tensorflow:global_step/sec: 389.308
INFO:tensorflow:loss = 3.71001e+10, step = 8601 (0.257 sec)
INFO:tensorflow:global_step/sec: 371.633
INFO:tensorflow:loss = 5.2148e+10, step = 8701 (0.267 sec)
INFO:tensorflow:global_step/sec: 340.928
INFO:tensorflow:loss = 1.94383e+11, step = 8801 (0.291 sec)
INFO:tensorflow:global_step/sec: 359.162
INFO:tensorflow:loss = 7.08984e+10, step = 8901 (0.279 sec)
INFO:tensorflow:global_step/sec: 33

INFO:tensorflow:loss = 6.71272e+10, step = 16101 (0.260 sec)
INFO:tensorflow:global_step/sec: 391.909
INFO:tensorflow:loss = 1.15117e+11, step = 16201 (0.256 sec)
INFO:tensorflow:global_step/sec: 444.883
INFO:tensorflow:loss = 4.55799e+10, step = 16301 (0.224 sec)
INFO:tensorflow:global_step/sec: 465.446
INFO:tensorflow:loss = 1.26335e+11, step = 16401 (0.215 sec)
INFO:tensorflow:global_step/sec: 384.68
INFO:tensorflow:loss = 4.59799e+10, step = 16501 (0.262 sec)
INFO:tensorflow:global_step/sec: 413.822
INFO:tensorflow:loss = 5.75461e+10, step = 16601 (0.244 sec)
INFO:tensorflow:global_step/sec: 500.395
INFO:tensorflow:loss = 7.54595e+10, step = 16701 (0.198 sec)
INFO:tensorflow:global_step/sec: 366.049
INFO:tensorflow:loss = 2.12238e+10, step = 16801 (0.274 sec)
INFO:tensorflow:global_step/sec: 368.41
INFO:tensorflow:loss = 1.3378e+11, step = 16901 (0.271 sec)
INFO:tensorflow:global_step/sec: 403.665
INFO:tensorflow:loss = 5.49092e+10, step = 17001 (0.248 sec)
INFO:tensorflow:global_s

INFO:tensorflow:global_step/sec: 446.108
INFO:tensorflow:loss = 8.88964e+10, step = 24201 (0.226 sec)
INFO:tensorflow:global_step/sec: 416.592
INFO:tensorflow:loss = 6.32719e+10, step = 24301 (0.237 sec)
INFO:tensorflow:global_step/sec: 368.168
INFO:tensorflow:loss = 7.35571e+10, step = 24401 (0.273 sec)
INFO:tensorflow:global_step/sec: 416.502
INFO:tensorflow:loss = 1.41199e+11, step = 24501 (0.241 sec)
INFO:tensorflow:global_step/sec: 416.086
INFO:tensorflow:loss = 8.39407e+10, step = 24601 (0.241 sec)
INFO:tensorflow:global_step/sec: 410.939
INFO:tensorflow:loss = 5.63831e+10, step = 24701 (0.243 sec)
INFO:tensorflow:global_step/sec: 378.51
INFO:tensorflow:loss = 9.64357e+10, step = 24801 (0.261 sec)
INFO:tensorflow:global_step/sec: 351.515
INFO:tensorflow:loss = 6.64292e+10, step = 24901 (0.284 sec)
INFO:tensorflow:Saving checkpoints for 25000 into /var/folders/jr/44cwffjj7nj68zjg2ngbdmjw0000gn/T/tmp85a_z68z/model.ckpt.
INFO:tensorflow:Loss for final step: 4.98389e+10.


** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [39]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [41]:
pred_gen=dnn_model.predict(predict_input_func)

In [42]:
predictions= list(pred_gen)

INFO:tensorflow:Restoring parameters from /var/folders/jr/44cwffjj7nj68zjg2ngbdmjw0000gn/T/tmp85a_z68z/model.ckpt-25000


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [43]:
from sklearn.metrics import mean_squared_error

In [44]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [45]:
mean_squared_error(y_test,final_preds)**0.5

90585.951765131089

97921.93181985477

# Great Job!